# Create a Service to transcrbe the earnings call
The whisper service has already transcribed sound files relating to the earnings calls.  It was hosted on a snowpark container and it processed each file with a medium sized GPU.  Whisper leverages pytorch (a deap leaning framework) to parse and transcribe the text.  If you want to set this up yourself, follow the instructions here to set up the container in Snowflake.

- git hub https://github.com/michaelgorkow/scs_whisper, 
- blog post https://github.com/michaelgorkow/scs_whisper

There is also a quick start which also leverages the Whisper Service using a **Container Runtime Notebook**

- https://quickstarts.snowflake.com/guide/ai_agent_health_payers_cc/index.html?index=..%2F..index#0

For today, we will use the transcripts which were processed in advance of this lab.  For reference, the results are shared to you via the **Internal Marketplace**

####  Listen to the calls that were transcribed

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


files = session.sql('''SELECT RELATIVE_PATH, GET_PRESIGNED_URL('@DOCUMENT_AI.EARNINGS_CALLS',RELATIVE_PATH) URL FROM DIRECTORY (@DOCUMENT_AI.EARNINGS_CALLS)''')


select_call = st.selectbox('Select Call:', files.select('RELATIVE_PATH'))

URL = files.filter(col('RELATIVE_PATH') == select_call).select('URL').collect()[0][0]
st.audio(URL, format="audio/mpeg")

#### The sound files have already been transcribed using the whisper service.
We will now create a table based on the raw results produced by the Whisper Service. 

In [ ]:
SELECT * FROM DEFAULT_SCHEMA.EARNINGS_CALL_TRANSCRIPT

### Transform the transcript table
We are creating a table which is flattening the json data that was generated by whisper.  This creates a row for every snippet of commentary.

In [ ]:
CREATE TABLE IF NOT EXISTS DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS AS

SELECT RELATIVE_PATH, PARSE_JSON(TRANSCRIPT):language::TEXT LANGUAGE,

VALUE:end::FLOAT TIME_SECONDS,  
VALUE:text::TEXT TEXT 
FROM DEFAULT_SCHEMA.EARNINGS_CALL_TRANSCRIPT,
LATERAL FLATTEN (PARSE_JSON(TRANSCRIPT):segments);

SELECT * FROM DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS LIMIT 5

#### Add Sentiment scores to the calls
Here, Cortex Sentiment is being used to generate a sentiment score for each text snippet

In [ ]:
SELECT *, SNOWFLAKE.CORTEX.SENTIMENT(TEXT) FROM DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS

#### Put all together in Streamlit
Let's now have a look at the results using a visualistion in Streamlit

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

def sentiment(text):
    return call_function('snowflake.cortex.sentiment',text)

transcript_with_sentiment = session.table('DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS').with_column('sentiment',sentiment(col('TEXT')))

st.markdown('#### Calls with Sentiment')


st.dataframe(transcript_with_sentiment)
col1,col2,col3 = st.columns(3)

with col1:

    st.markdown('#### Q1')
    q1 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q1_FY2025.mp3')
    st.line_chart(q1,
              y='SENTIMENT',x='TIME_SECONDS',color = '#29B5E8')
    st.metric('Average Sentiment',q1.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col2:
    q2 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q2_FY2025.mp3')
    st.markdown('#### Q2')
    st.line_chart(q2,
              y='SENTIMENT',x='TIME_SECONDS',color = '#29B5E8')
    st.metric('Average Sentiment',q2.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col3:
    
    st.markdown('#### Q3')
    q3 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q3_FY2025.mp3')
    st.line_chart(q3,
              y='SENTIMENT',x='TIME_SECONDS',color = '#FF9F36')
    st.metric('Average Sentiment',q3.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

You will see that the line charts are very hard to read.  Also with such a small snippet of information, it is difficult to get a good sentiment score.  Unlike the chunking which you did with the Analyst reports, this time we are going to group snippets of data together for every 60 seconds and then create an average sentiment for every minute. 

In [ ]:
grouped = transcript_with_sentiment.with_column('TIME',time_from_parts(15,0,'TIME_SECONDS')).\
with_column('MINUTES',date_trunc('minute','TIME'))
grouped = grouped.with_column('MINUTES',minute('MINUTES'))
data_grouped_minutes = grouped.group_by('RELATIVE_PATH','MINUTES').agg(array_agg('TEXT').alias('TEXT'),avg('SENTIMENT').alias('SENTIMENT'))

st.markdown('''#### Data Grouped to Minutes''')
data_grouped_minutes


Below is a much smoother output - and a lot easier to read.  We are also able to view these grouped snippets in the visualistion.  Here, we are featuring the Most populate minute of the year, followed by the most negative minute of the year.

In [ ]:
st.markdown('#### Sentiment Analysis during the duration of the last 3 quarterly earnings calls')
col1,col2,col3 = st.columns(3)

with col1:

    st.markdown('#### Q1')
    q1 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q1_FY2025.mp3')
    st.line_chart(q1,
              y='SENTIMENT',x='MINUTES',color = '#29B5E8')
    st.metric('Average Sentiment',q1.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col2:
    q2 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q2_FY2025.mp3')
    st.markdown('#### Q2')
    st.line_chart(q2,
              y='SENTIMENT',x='MINUTES',color = '#29B5E8')
    st.metric('Average Sentiment',q2.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col3:
    
    st.markdown('#### Q3')
    q3 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q3_FY2025.mp3')
    st.line_chart(q3,
              y='SENTIMENT',x='MINUTES',color = '#FF9F36')
    st.metric('Average Sentiment',q3.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

st.markdown(f'''**:bulb: Most positive minute of the year**: \
{data_grouped_minutes.sort(col('SENTIMENT').desc()).limit(1).select(array_to_string(col('TEXT'),lit(''))).collect()[0][0]}''')

st.markdown(f'''**:warning: Most negative minute of the year**: \
{data_grouped_minutes.sort(col('SENTIMENT').asc()).limit(1).select(array_to_string(col('TEXT'),lit(''))).collect()[0][0]}''')

Now lets remove the arrays to visualise pure text.

In [ ]:
grouped_text = data_grouped_minutes.with_column(
    'TEXT',
    regexp_replace(cast(col('TEXT'), StringType()), r'[\[\]"]', '')
)
grouped_text

#### Save data in a table
Finally lets save the result in a table.  Once this is complete, move to **Step 6** - Create a Search Service

In [ ]:
grouped_text.write.mode("overwrite").save_as_table("DEFAULT_SCHEMA.summary_text")